<a href="https://colab.research.google.com/github/Ariotegar/CitraKomp/blob/main/Webapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CELL 1:Install required dependencies

In [1]:
!pip install streamlit opencv-python-headless torch torchvision numpy pillow pyngrok
!pip install yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 9.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=0872be61754460049928dbd2838fc8227f9eac2a51f5f43364b5f28e0f30a432
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b1

# CELL 2: Configure ngrok authentication

In [4]:
from pyngrok import ngrok
ngrok.set_auth_token("API_KEY")

# CELL 3: Create the app.py file

In [2]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
import torch
from PIL import Image
import io

def load_yolov5_model():
    """
    Load YOLOv5 model with the best weights
    """
    model = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt')
    return model

def process_image(image, operation):
    """
    Apply various image processing operations
    """
    # Convert PIL Image to numpy array
    img_array = np.array(image)

    if operation == "Grayscale":
        processed = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
        return Image.fromarray(processed)

    elif operation == "Blur":
        processed = cv2.GaussianBlur(img_array, (5,5), 0)
        return Image.fromarray(processed)

    elif operation == "Edge Detection":
        gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
        processed = cv2.Canny(gray, 100, 200)
        return Image.fromarray(processed)

    elif operation == "Sharpen":
        kernel = np.array([[-1,-1,-1],
                         [-1, 9,-1],
                         [-1,-1,-1]])
        processed = cv2.filter2D(img_array, -1, kernel)
        return Image.fromarray(processed)

    return image

def detect_objects(image, model):
    # Make prediction
    results = model(image)

    # Convert results to DataFrame
    df = results.pandas().xyxy[0]

    return results, df

def main():
    st.title("Image Processing and Object Detection App")

    # Sidebar for operations selection
    st.sidebar.title("Operations")
    operation = st.sidebar.selectbox(
        "Select Operation",
        ["Original", "Grayscale", "Blur", "Edge Detection", "Sharpen", "Object Detection"]
    )

    # File uploader
    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        # Load and display original image
        image = Image.open(uploaded_file)
        st.image(image, caption="Original Image", use_column_width=True)

        if operation != "Original" and operation != "Object Detection":
            # Process image
            processed_image = process_image(image, operation)
            st.image(processed_image, caption=f"Processed Image ({operation})", use_column_width=True)

        elif operation == "Object Detection":
            try:
                # Load YOLOv5 model
                model = load_yolov5_model()

                # Perform detection
                results, df = detect_objects(image, model)

                # Display results
                results.render()
                detected_img = Image.fromarray(results.ims[0])
                st.image(detected_img, caption="Detected Objects", use_column_width=True)

                # Display detection details
                st.write("Detection Results:")
                st.dataframe(df[['name', 'confidence']].round(2))

            except Exception as e:
                st.error(f"Error in object detection: {str(e)}")

if __name__ == "__main__":
    main()

Overwriting app.py


# CELL 5: Upload YOLOv5 Model dan Run Streamlit with ngrok

In [6]:
from google.colab import files
uploaded = files.upload()

!nohup streamlit run app.py &
print("Launching Streamlit app...")

import time
time.sleep(5)

try:
    public_url = ngrok.connect(8501)
    print(f"Public URL: {public_url}")
except Exception as e:
    print(f"Error: {str(e)}")

Saving CitraKomp.pt to CitraKomp (2).pt
nohup: appending output to 'nohup.out'
Launching Streamlit app...
Public URL: NgrokTunnel: "https://a240-34-125-64-199.ngrok-free.app" -> "http://localhost:8501"
